In [ ]:
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_dense_adj

# Generating a PPI-style graph (1,603 nodes, 2,757 edges) as per paper specs
num_nodes = 1603
num_edges = 2757
g = nx.gnm_random_graph(num_nodes, num_edges, seed=42)
adj = nx.adjacency_matrix(g).todense()
A = torch.tensor(adj).float()